In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, datetime
import holidays
v_directory = 'Artigo/Analise de Sentimento/TimeSeriesArticle'

# Analysis for S&P500 index

### Are there missing values on the original data?

In [2]:
v_sp500 = pd.read_csv('S&P500.csv', parse_dates=True)
v_sp500.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
5079,2020-03-12,2630.860107,2660.949951,2478.860107,2480.639893,2480.639893,8829380000
5080,2020-03-13,2569.989990,2711.330078,2492.370117,2711.020020,2711.020020,8258670000
5081,2020-03-16,2508.590088,2562.979980,2380.939941,2386.129883,2386.129883,7781540000
5082,2020-03-17,2425.659912,2553.929932,2367.040039,2529.189941,2529.189941,8358500000
5083,2020-03-18,2436.500000,2453.570068,2280.520020,2398.100098,2398.100098,8755780000


In [3]:
len(v_sp500)

5084

In [4]:
at_least_one_value_missing = v_sp500.apply(lambda x: (len(x)-x.count())>=1, axis=1)

In [5]:
at_least_one_value_missing.value_counts()

False    5084
dtype: int64

#### There are no missing values on the original data.
### Are there missing days on the original data?

In [6]:
business_dates = pd.bdate_range(start='2000-01-03', end='2020-03-18').date
business_dates = business_dates.tolist()

In [7]:
num_business_days = len(business_dates)
num_business_days

5273

In [8]:
difference = num_business_days-len(v_sp500)
difference

189

#### There are 189 missing business days on the data.
### Are those holidays? 
#### It is possible, since there are 20 years of data and approximately 10 holidays per year in the USA, and a few might have happened in weekends (that are already not being considered)

In [9]:
dates = list()
years = list(range(2000,2021))
us_holidays = holidays.UnitedStates(years = years, expand = False, observed=False)
#us_holidays

In [10]:
us_holidays[datetime(2019,4,19)] = 'Good Friday'
us_holidays[datetime(2018,12,5)] = 'Mourning Bush'
us_holidays[datetime(2018,3,30)] = 'Good Friday'
us_holidays[datetime(2017,4,14)] = 'Good Friday'
us_holidays[datetime(2017,1,2)] = "New Year's"
us_holidays[datetime(2016,3,25)] = 'Good Friday'
us_holidays[datetime(2016,12,26)] = 'Christmas'
us_holidays[datetime(2015,7,3)] = 'Independence Day'
us_holidays[datetime(2015,4,3)] = 'Good Friday'
us_holidays[datetime(2019,4,19)] = 'Good Friday'
us_holidays[datetime(2014,4,18)] = 'Good Friday'
us_holidays[datetime(2013,3,29)] = 'Good Friday'
us_holidays[datetime(2012,10,30)] = 'Hurricane Sandy'
us_holidays[datetime(2012,10,29)] = 'Hurricane Sandy'
us_holidays[datetime(2012,4,6)] = 'Good Friday'
us_holidays[datetime(2012,1,2)] = "New Year's"
us_holidays[datetime(2011,12,26)] = 'Christmas'
us_holidays[datetime(2011,4,22)] = 'Good Friday'
us_holidays[datetime(2010,12,24)] = 'Christmas'
us_holidays[datetime(2010,7,5)] = 'Independence Day'
us_holidays[datetime(2010,4,2)] = 'Good Friday'
us_holidays[datetime(2009,7,3)] = 'Independence Day'
us_holidays[datetime(2009,4,10)] = 'Good Friday'
us_holidays[datetime(2008,3,21)] = 'Good Friday'
us_holidays[datetime(2007,4,6)] = 'Good Friday'
us_holidays[datetime(2007,1,2)] = 'Mourning Ford'
us_holidays[datetime(2006,4,14)] = 'Good Friday'
us_holidays[datetime(2006,1,2)] = "New Year's"
us_holidays[datetime(2005,12,26)] = 'Christmas'
us_holidays[datetime(2005,3,25)] = 'Good Friday'
us_holidays[datetime(2004,12,24)] = 'Christmas'
us_holidays[datetime(2004,7,5)] = 'Independence Day'
us_holidays[datetime(2004,6,11)] = 'Mourning Reagan'
us_holidays[datetime(2004,4,9)] = 'Good Friday'
us_holidays[datetime(2003,4,18)] = 'Good Friday'
us_holidays[datetime(2002,3,29)] = 'Good Friday'
us_holidays[datetime(2001,9,14)] = '9-11'
us_holidays[datetime(2001,9,13)] = '9-11'
us_holidays[datetime(2001,9,12)] = '9-11'
us_holidays[datetime(2001,9,11)] = '9-11'
us_holidays[datetime(2001,4,13)] = 'Good Friday'
us_holidays[datetime(2000,4,21)] = 'Good Friday'


In [11]:
for date in business_dates:
    if(date not in list(us_holidays.keys())):
        dates.append(date)
        #print("Not Holiday", date) 
    else:
        print("Holiday", date) 

Holiday 2000-01-17
Holiday 2000-02-21
Holiday 2000-04-21
Holiday 2000-05-29
Holiday 2000-07-04
Holiday 2000-09-04
Holiday 2000-10-09
Holiday 2000-11-23
Holiday 2000-12-25
Holiday 2001-01-01
Holiday 2001-01-15
Holiday 2001-02-19
Holiday 2001-04-13
Holiday 2001-05-28
Holiday 2001-07-04
Holiday 2001-09-03
Holiday 2001-09-11
Holiday 2001-09-12
Holiday 2001-09-13
Holiday 2001-09-14
Holiday 2001-10-08
Holiday 2001-11-22
Holiday 2001-12-25
Holiday 2002-01-01
Holiday 2002-01-21
Holiday 2002-02-18
Holiday 2002-03-29
Holiday 2002-05-27
Holiday 2002-07-04
Holiday 2002-09-02
Holiday 2002-10-14
Holiday 2002-11-11
Holiday 2002-11-28
Holiday 2002-12-25
Holiday 2003-01-01
Holiday 2003-01-20
Holiday 2003-02-17
Holiday 2003-04-18
Holiday 2003-05-26
Holiday 2003-07-04
Holiday 2003-09-01
Holiday 2003-10-13
Holiday 2003-11-11
Holiday 2003-11-27
Holiday 2003-12-25
Holiday 2004-01-01
Holiday 2004-01-19
Holiday 2004-02-16
Holiday 2004-04-09
Holiday 2004-05-31
Holiday 2004-06-11
Holiday 2004-07-05
Holiday 2004

In [12]:
len(dates)

5051

In [13]:
num_business_days-len(dates)

222

In [14]:
len(v_sp500)-len(dates)

33

### Dates present in sp500 but not on our list date

In [15]:
count = 0
for date in v_sp500['Date']:
    if datetime.strptime(date, '%Y-%m-%d').date() not in dates:
        print(datetime.strptime(date, '%Y-%m-%d').date())
        count +=1
         
print(count, "dates are missing from our dates list")

2000-10-09
2001-10-08
2002-10-14
2002-11-11
2003-10-13
2003-11-11
2004-10-11
2004-11-11
2005-10-10
2005-11-11
2006-10-09
2007-10-08
2008-10-13
2008-11-11
2009-10-12
2009-11-11
2010-10-11
2010-11-11
2011-10-10
2011-11-11
2012-10-08
2013-10-14
2013-11-11
2014-10-13
2014-11-11
2015-10-12
2015-11-11
2016-10-10
2016-11-11
2017-10-09
2018-10-08
2019-10-14
2019-11-11
33 dates are missing from our dates list


#### they are all holidays that still had the index informed. No problem here.


### Dates present on our date list but missing from sp500

In [16]:
count = 0
for date in dates:
    if date.strftime('%Y-%m-%d') not in v_sp500['Date'].unique():
        print(date.strftime('%Y-%m-%d'))
        count += 1
print(count, " dates missing from sp500 list")

0  dates missing from sp500 list


#### The dataset is complete

# Analysis for vix index

### Are there missing values on the original data?

In [17]:
v_vix = pd.read_csv('VIX.csv', parse_dates=True)
v_vix.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
5079,2020-03-12,61.459999,76.830002,59.910000,75.470001,75.470001,0
5080,2020-03-13,65.070000,77.570000,55.169998,57.830002,57.830002,0
5081,2020-03-16,57.830002,83.559998,57.830002,82.690002,82.690002,0
5082,2020-03-17,82.690002,84.830002,70.370003,75.910004,75.910004,0
5083,2020-03-18,69.370003,85.470001,69.370003,76.449997,76.449997,0


In [18]:
len(v_vix)

5084

In [19]:
at_least_one_value_missing = v_vix.apply(lambda x: (len(x)-x.count())>=1, axis=1)

In [20]:
at_least_one_value_missing.value_counts()

False    5084
dtype: int64

#### There are no missing values on the original data.
### Are there missing days on the original data?

In [21]:
num_business_days = len(business_dates)
num_business_days

5273

In [22]:
difference = num_business_days-len(v_vix)
difference

189

#### There are 189 missing business days on the data.
### Are those holidays? 
#### It is possible, since there are 20 years of data and approximately 10 holidays per year in the USA, and a few might have happened in weekends (that are already not being considered)

In [23]:
dates = list()
for date in business_dates:
    if(date not in list(us_holidays.keys())):
        dates.append(date)
        #print("Not Holiday", date) 
    else:
        print("Holiday", date) 

Holiday 2000-01-17
Holiday 2000-02-21
Holiday 2000-04-21
Holiday 2000-05-29
Holiday 2000-07-04
Holiday 2000-09-04
Holiday 2000-10-09
Holiday 2000-11-23
Holiday 2000-12-25
Holiday 2001-01-01
Holiday 2001-01-15
Holiday 2001-02-19
Holiday 2001-04-13
Holiday 2001-05-28
Holiday 2001-07-04
Holiday 2001-09-03
Holiday 2001-09-11
Holiday 2001-09-12
Holiday 2001-09-13
Holiday 2001-09-14
Holiday 2001-10-08
Holiday 2001-11-22
Holiday 2001-12-25
Holiday 2002-01-01
Holiday 2002-01-21
Holiday 2002-02-18
Holiday 2002-03-29
Holiday 2002-05-27
Holiday 2002-07-04
Holiday 2002-09-02
Holiday 2002-10-14
Holiday 2002-11-11
Holiday 2002-11-28
Holiday 2002-12-25
Holiday 2003-01-01
Holiday 2003-01-20
Holiday 2003-02-17
Holiday 2003-04-18
Holiday 2003-05-26
Holiday 2003-07-04
Holiday 2003-09-01
Holiday 2003-10-13
Holiday 2003-11-11
Holiday 2003-11-27
Holiday 2003-12-25
Holiday 2004-01-01
Holiday 2004-01-19
Holiday 2004-02-16
Holiday 2004-04-09
Holiday 2004-05-31
Holiday 2004-06-11
Holiday 2004-07-05
Holiday 2004

In [24]:
len(dates)

5051

In [25]:
num_business_days-len(dates)

222

In [26]:
len(v_sp500)-len(dates)

33

### Dates present in vix but not on our list date

In [27]:
count = 0
for date in v_vix['Date']:
    if datetime.strptime(date, '%Y-%m-%d').date() not in dates:
        print(datetime.strptime(date, '%Y-%m-%d').date())
        count +=1
         
print(count, "dates are missing from our dates list")

2000-10-09
2001-10-08
2002-10-14
2002-11-11
2003-10-13
2003-11-11
2004-10-11
2004-11-11
2005-10-10
2005-11-11
2006-10-09
2007-10-08
2008-10-13
2008-11-11
2009-10-12
2009-11-11
2010-10-11
2010-11-11
2011-10-10
2011-11-11
2012-10-08
2013-10-14
2013-11-11
2014-10-13
2014-11-11
2015-10-12
2015-11-11
2016-10-10
2016-11-11
2017-10-09
2018-10-08
2019-10-14
2019-11-11
33 dates are missing from our dates list


#### they are all holidays that still had the index informed. No problem here.


### Dates present on our date list but missing from vix

In [28]:
count = 0
for date in dates:
    if date.strftime('%Y-%m-%d') not in v_vix['Date'].unique():
        print(date.strftime('%Y-%m-%d'))
        count += 1
print(count, " dates missing from vix list")

0  dates missing from vix list


#### The dataset is complete.

# Analysis for EMV index

In [29]:
v_monthly_EMV_index =  pd.read_excel('EMV_Data.xlsx') 
v_monthly_EMV_index = v_monthly_EMV_index[:-1]
v_monthly_EMV_index.head()

,Year,Month,Overall EMV Tracker,Policy-Related EMV Tracker,Infectious Disease EMV Tracker,Macroeconomic News and Outlook EMV Tracker,Macro – Broad Quantity Indicators EMV Tracker,Macro – Inflation EMV Indicator,Macro – Interest Rates EMV Tracker,Macro – Other Financial Indicators EMV Tracker,...,Housing and Land Management EMV Tracker,Other Regulation EMV Tracker,National Security Policy EMV Tracker,Government-Sponsored Enterprises EMV Tracker,Trade Policy EMV Tracker,Healthcare Policy EMV Tracker,Food and Drug Policy EMV Tracker,"Transportation, Infrastructure, and Public Utilities EMV Tracker",Elections and Political Governance EMV Tracker,Agricultural Policy EMV Tracker
0,1985,1,11.304017,5.397642,0.322972,8.074298,2.018574,4.037149,4.037149,0.645944,...,0.080743,0.080743,1.049659,0.322972,0.080743,0.484458,0.080743,0.242229,0.161486,0.080743
1,1985,2,9.461739,4.269774,0.323478,7.278260,1.617391,2.911304,2.911304,0.404348,...,0.080870,0.161739,2.345217,0.323478,0.080870,0.970435,0.000000,0.242609,0.323478,0.080870
2,1985,3,10.997636,4.873834,0.578823,7.524698,2.149914,3.721005,3.638316,0.248067,...,0.082689,0.413445,0.744201,0.578823,0.165378,0.248067,0.000000,0.000000,0.082689,0.082689
3,1985,4,9.267448,4.496748,0.142576,7.342671,1.924778,2.423794,3.920844,0.356440,...,0.000000,0.356440,0.712881,0.570305,0.142576,0.071288,0.000000,0.285152,0.142576,0.000000
4,1985,5,9.836281,4.316045,0.330077,7.459730,2.574597,2.574597,3.960918,0.396092,...,0.066015,0.264061,0.924214,0.462107,0.132031,0.198046,0.132031,0.462107,0.132031,0.000000


In [30]:
at_least_one_value_missing = v_monthly_EMV_index.apply(lambda x: (len(x)-x.count())>=1, axis=1)

In [31]:
at_least_one_value_missing.value_counts()

False    424
dtype: int64

In [32]:
v_monthly_EMV_index.shape

(424, 46)

#### There are no missing values for the emv indexes, but are there missing months?

In [33]:
print(v_monthly_EMV_index.iloc[0]['Year'])
print(v_monthly_EMV_index.iloc[0]['Month'])

1985.0
1.0


In [34]:
print(v_monthly_EMV_index.iloc[-1]['Year'])
print(v_monthly_EMV_index.iloc[-1]['Month'])

2020.0
4.0


In [35]:
num_years = v_monthly_EMV_index.iloc[-1]['Year'] - v_monthly_EMV_index.iloc[0]['Year']
num_years

35.0

In [36]:
num_months = num_years*12 -(v_monthly_EMV_index.iloc[0]['Month']-1) + v_monthly_EMV_index.iloc[-1]['Month']
num_months

424.0

#### Apperently everything is ok, since the number of registries and the number of months between the dates are the same